# Create Pipeline Package for the soft sensor use case

In this notebook, the main goal is to create a pipeline with all of the contents that are necessary for the execution of the model on an Industrial Edge device.  
In order to put the elements together, this example collects files from the previous notebooks.


### Creating a PythonComponent

In this step, we create a `PythonComponent` that executes the Python script we created in [notebook 20](./20-CreateInferenceWrapper.ipynb).\
To do so, we need to 
- collect our file resources
- define component inputs and outputs
- define the required Python environment

The most important information of a PythonComponent is the Python version which the model and Python scripts can work, and it must be set in initialization time.  
The further information should be added are the name and a little description on the component.

In [8]:
from simaticai import deployment

component = deployment.PythonComponent(
    name='valve_controller',  # name of the Pipeline Step
    python_version='3.10',    # Python version
    desc="Predicts the control action based on the sensor payload"  # Short description of the Pipeline Step
)

## Adding file resources

As we discussed in [notebook 20](./20-CreateInferenceWrapper.ipynb), the component will predict with the created model and the python code we implemented will execute that prediction.  
We created the [entrypoint.py](../src/inference/entrypoint.py) Python file which implements the required `process_input(..)` method.  
To create the component, we need to add those file resources and set the entrypoint.

Note that the path to the `entrypoint.py` file is `../src/inference/entrypoint.py`.  
Similarly, the path to our model is `../models/model.joblib`.

In [9]:
component.add_resources(base_dir='../src/inference', resources='entrypoint.py')
component.add_resources(base_dir='../models', resources='model.joblib')
component.set_entrypoint('entrypoint.py')

## Defining component inputs

The inputs on the Component must be set to inform AI Inference Server which variables should be gathered from the different data sources and we want to work with them in our entrypoint.  
In our example, these are the temperature and valve position values.  
In this case we also want to configure that data will be received in batches, so we configure the `inputBatch` on the component to `True`.

In [10]:
component.add_input("temperature_A", "Double")
component.add_input("temperature_B", "Double")
component.add_input("temperature_C", "Double")
component.add_input("valve_position_A", "Double")
component.add_input("valve_position_B", "Double")

component.batch.inputBatch = True

## Defining component outputs

Similarly to the inputs, we also need to inform the AI Inference Server which data will be provided by our component.\
It is our responsibility to provide these data in a dictionary with the keys that reflects on the output variable name.

In [11]:
# as we defined in the entrypoint.py we return the below dictionary
# [...]
# return {"valve_position_A": valve_position_A, "valve_position_B": valve_position_B, "predicted_phC": phC_predictions.mean()}

component.add_output("predicted_phC", "Double")
component.add_output("valve_control_A", "Double")
component.add_output("valve_control_B", "Double")

### Defining Python dependencies

The `requirements.txt` should contain only those dependencies which are required while running the model and inference wrapper on AI Inference Server.  
Dependencies used during model creation or pipeline building should be excluded.  
For this reason we collected all necessary dependencies in the file `runtime_requirements.txt`.


In [ ]:
component.set_requirements('../src/inference/runtime_requirements.txt')

### Creating a pipeline from the component

Now we can use the component to create a pipeline configuration.  
The pipeline requires a list of components - which is a single component in this example - and a name.

In [ ]:
pipeline = deployment.Pipeline.from_components(
    [component], 
    name='SoftSensorPipeline',
    desc='Pipeline to predict the control action based on the sensor payload'
)
pipeline

### Build the Pipeline Package

This step creates the proper content in the defined target folder `packages` and creates the edge Pipeline Package as a zip file.  
The export method first validates the pipeline and raises an error if it finds any problems.


In [ ]:
edge_package_path = pipeline.export(destination = '../packages')
edge_package_path

### Test the Pipeline Package locally

We suggest to test the Pipeline Package on your computer before deploying it to an Edge device. It is possible to do so using notebook [40-TestPipelineLocally.ipynb](40-TestPipelineLocally.ipynb).